In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

pd.set_option('display.max_columns', None)


In [2]:
df = pd.read_csv("data.csv", error_bad_lines=False, header=0, sep="\t")
df.drop([
    "Unnamed: 0",
    "index",
    "thumbnail",
    "externalReference",
    "operation",
    "address",
    "province",
    "municipality",
    "country",
    "url",
    "distance",
    "suggestedTexts",
    "detailedType",
    "status",
    "newDevelopment",
    "topNewDevelopment",
    "neighborhood",
    "priceByArea",
    "latitude",
    "longitude"
], axis=1, inplace=True)

df.head(3)

,propertyCode,numPhotos,floor,price,propertyType,size,exterior,rooms,bathrooms,district,showAddress,hasVideo,hasLift,hasPlan,has3DTour,has360,parkingSpace
0,89896113,22,3,680.0,flat,64.0,False,3,1,Villaverde,True,False,False,True,False,False,NaN
1,89716445,22,3,680.0,flat,64.0,False,3,1,Villaverde,True,True,False,True,False,True,NaN
2,89853301,1,bj,580.0,flat,35.0,True,1,1,Villaverde,False,False,False,False,False,False,NaN


In [3]:
df_x = df.drop("price", axis=1)
df_y = df["price"]

df["hasLift"] = df["hasLift"].astype(bool)

In [4]:
df["isParkingSpaceIncludedInPrice"] = False
df["hasParkingSpace"] = False

for index, rows in df.iterrows():
    try:
        df.loc[index, ["isParkingSpaceIncludedInPrice"]] =  eval(rows["parkingSpace"]).get("isParkingSpaceIncludedInPrice")
        df.loc[index, ["hasParkingSpace"]] =  eval(rows["parkingSpace"]).get("hasParkingSpace")
    except:
        pass

df.drop("parkingSpace", axis=1, inplace=True)


In [5]:
df.corr()["price"].sort_values()

exterior                        -0.466109
has3DTour                       -0.068806
propertyCode                    -0.018524
showAddress                      0.068955
has360                           0.077445
hasParkingSpace                  0.134041
hasPlan                          0.142339
isParkingSpaceIncludedInPrice    0.151017
hasVideo                         0.154974
hasLift                          0.207176
numPhotos                        0.351007
rooms                            0.708376
bathrooms                        0.786479
size                             0.887713
price                            1.000000
Name: price, dtype: float64

In [6]:
df.dtypes

propertyCode                       int64
numPhotos                          int64
floor                             object
price                            float64
propertyType                      object
size                             float64
exterior                            bool
rooms                              int64
bathrooms                          int64
district                          object
showAddress                         bool
hasVideo                            bool
hasLift                             bool
hasPlan                             bool
has3DTour                           bool
has360                              bool
isParkingSpaceIncludedInPrice       bool
hasParkingSpace                     bool
dtype: object

In [7]:
# sustituir "baj" por 0 y tramear

df.loc[df["floor"] == "bj", ["floor"]] = 0
df.loc[df["floor"] == "1", ["floor"]] = 1
df.loc[df["floor"] == "2", ["floor"]] = 2

df.astype({"floor": float}, errors="ignore")
df.loc[~df["floor"].isin([0,1,2]), ["floor"]] = "3+"
df["floor"].value_counts()

3+    199
2     100
1      97
0      54
Name: floor, dtype: int64

In [8]:
df["propertyType"].value_counts() #a dummy

flat         278
chalet        98
duplex        32
penthouse     28
studio        14
Name: propertyType, dtype: int64

In [9]:
df["district"].value_counts() #a dummy

Encinar de los Reyes                   175
Villaverde                              45
Zona Prado de Somosaguas - La Finca     35
Barajas                                 34
Villa de Vallecas                       28
San Blas                                26
Zona norte                              25
Hortaleza                               20
Fuencarral                              15
Zona Pueblo                             15
Carabanchel                             13
Moncloa                                  8
Zona Avenida Europa                      4
Vicálvaro                                3
Latina                                   3
Zona Estación                            1
Name: district, dtype: int64

In [10]:
df_dummies = pd.get_dummies(df)

In [11]:
df_dummies.corr()["price"].sort_values()

propertyType_flat                              -0.468345
exterior                                       -0.466109
district_Villaverde                            -0.217198
district_Villa de Vallecas                     -0.162662
district_San Blas                              -0.146995
floor_1                                        -0.133540
district_Barajas                               -0.126237
propertyType_studio                            -0.118557
district_Carabanchel                           -0.109190
district_Fuencarral                            -0.104823
district_Zona Pueblo                           -0.104025
floor_2                                        -0.103201
floor_0                                        -0.086321
district_Hortaleza                             -0.085370
has3DTour                                      -0.068806
district_Latina                                -0.046649
district_Zona Avenida Europa                   -0.036118
district_Vicálvaro             

In [12]:
X = df_dummies.drop("price", axis=1)
y = df_dummies["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [13]:
reg = LinearRegression().fit(X_train, y_train)
prediction = reg.predict(X_test)
r2_score(y_test, prediction)

0.8341358419300042